In [3]:
import pandas as pd
import string

In [4]:
kor_en_df=pd.read_csv('./kor_en.csv',header=None)
kor_en_df.columns=['english','korean']
kor_en_df.english=kor_en_df.english.apply(lambda x: x.lower())
kor_en_df.english=kor_en_df.english.apply(lambda x: "".join([s for s in x if s not in string.punctuation]) )
kor_en_df.korean=kor_en_df.korean.apply(lambda x: "".join([s for s in x if s not in string.punctuation]) )
kor_en_df.to_csv('kor_en.txt',sep='\t',header=False,index=False)

In [5]:
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import random
import tqdm
from torch.optim.adam import Adam

In [6]:

def get_BOW(corpus):
  BOW={"<SOS>":0, "<EOS>":1}

  for line in corpus:
    for word in line.split():
      if word not in BOW.keys():
        BOW[word]=len(BOW.keys())

  return BOW

In [7]:
eng_corpus=[]
kor_corpus=[]
path='./kor_en.txt'
with open(path,encoding='utf-8') as f:
  lines=f.read().split('\n')
  for line in lines:
    txt="".join(s for s in line if s not in string.punctuation).lower()
    split_txt=txt.split('\t')

    if len(split_txt)>=2:
      engtxt=split_txt[0]
      kortxt=split_txt[1]
      if (len(engtxt.split())<=10) & (len(kortxt.split())<=10) :
        eng_corpus.append(engtxt)
        kor_corpus.append(kortxt)

    else:
      split_word=txt.split(' ')
      if len(split_word)>=2:
        engtxt=split_word[0]
        kortxt=split_word[1]
        if (len(engtxt.split())<=10) & (len(kortxt.split())<=10) :
          eng_corpus.append(engtxt)
          kor_corpus.append(kortxt)

      else:
        print('something wrong')
        print(txt)
        continue

something wrong



In [8]:
class Eng2Kor(Dataset):
  def __init__(self,path):
    self.eng_corpus=[]
    self.kor_corpus=[]

    with open(path,encoding='utf-8') as f:
      lines=f.read().split('\n')
      for line in lines:
        txt="".join(s for s in line if s not in string.punctuation).lower()
        split_txt=txt.split('\t')

        if len(split_txt)>=2:
          engtxt=split_txt[0]
          kortxt=split_txt[1]
          if (len(engtxt.split())<=10) & (len(kortxt.split())<=10) :
            self.eng_corpus.append(engtxt)
            self.kor_corpus.append(kortxt)

        else:
          split_word=txt.split(' ')
          if len(split_word)>=2:
            engtxt=split_word[0]
            kortxt=split_word[1]
            if (len(engtxt.split())<=10) & (len(kortxt.split())<=10) :
              self.eng_corpus.append(engtxt)
              self.kor_corpus.append(kortxt)

          else:
            continue

    self.engBOW=get_BOW(self.eng_corpus)
    self.korBOW=get_BOW(self.kor_corpus)

  def gen_seq(self,line):
    seq=line.split()
    seq.append("<EOS>")
    return seq

  def __len__(self):
    return len(self.eng_corpus)

  def __getitem__(self,i):
    data=np.array([self.engBOW[txt] for txt in self.gen_seq(self.eng_corpus[i])])
    label=np.array([self.korBOW[txt] for txt in self.gen_seq(self.kor_corpus[i])])
    return data,label

  def loader(dataset):
    for i in range(len(dataset)):
      data,label=dataset[i]

      yield torch.tensor(data),torch.tensor(label)

In [9]:
class Encoder(nn.Module):
  def __init__(self,input_size,hidden_size):
    super(Encoder,self).__init__()

    self.embedding=nn.Embedding(input_size,hidden_size)
    self.gru=nn.GRU(hidden_size,hidden_size)

  def forward(self,x,h):
    x=self.embedding(x).view(1,1,-1)
    output, hidden = self.gru(x,h)
    return output, hidden

In [10]:
class Decoder(nn.Module):
  def __init__(self,hidden_size,output_size,dropout_p=0.1,max_length=11):
    super(Decoder,self).__init__()

    self.embedding=nn.Embedding(output_size,hidden_size)
    #어텐션 가중치 계산을 위한 MLP층
    self.attention=nn.Linear(hidden_size*2,max_length)
    #특징 추출을 위한 MLP층
    self.context=nn.Linear(hidden_size*2,hidden_size)
    self.dropout=nn.Dropout(dropout_p)

    #GRU
    self.gru=nn.GRU(hidden_size,hidden_size)
    self.out=nn.Linear(hidden_size,output_size)

    self.relu=nn.ReLU()
    self.softmax=nn.LogSoftmax(dim=1)

  def forward(self, x, h, encoder_outputs):
    x=self.embedding(x).view(1,1,-1)
    x=self.dropout(x)

    attn_weights=self.softmax(self.attention(torch.cat((x[0],h[0]),-1)))
    attn_applied=torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))

    output=torch.cat((x[0], attn_applied[0]),1)
    output=self.context(output).unsqueeze(0)
    output=self.relu(output)

    output,hidden=self.gru(output,h)
    output=self.out(output[0])

    return output


In [11]:
device='cuda' if torch.cuda.is_available() else 'cpu'
dataset=Eng2Kor('./kor_en.txt')

encoder=Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder=Decoder(64, len(dataset.korBOW),dropout_p=0.1).to(device)

encoder_optimizer=Adam(encoder.parameters(),lr=0.0001)
decoder_optimizer=Adam(decoder.parameters(),lr=0.0001)

for epoch in range(10):
  iterator=tqdm.tqdm(dataset.loader(),total=len(dataset))
  total_loss=0

  for data, label in iterator:
    data=torch.tensor(data,dtype=torch.long).to(device)
    label=torch.tensor(label,dtype=torch.long).to(device)

    #인코더의 초기 은닉 상태
    encoder_hidden=torch.zeros(1,1,64).to(device)
    #인코더의 모든 시점의 출력을 저장하는 변수
    encoder_outputs=torch.zeros(11,64).to(device)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    loss=0
    for ei in range(len(data)):
      encoder_output,encoder_hidden=encoder(data[ei],encoder_hidden)
      encoder_outputs[ei]=encoder_output[0,0] #인코더의 은닉상태를 저장

    decoder_input=torch.tensor([[0]]).to(device)

    decoder_hidden=encoder_hidden

    #티처 포싱 사용 여부를 랜덤하게 결정
    use_teacher_forcing=True if random.random()<0.5 else False

    if use_teacher_forcing:
      for di in range(len(label)):
        decoder_output=decoder(decoder_input,decoder_hidden,encoder_outputs)

        target=torch.tensor(label[di], dtype=torch.long).to(device)
        target=torch.unsqueeze(target,dim=0).to(device)
        loss += nn.CrossEntropyLoss()(decoder_output,target)
        decoder_input=target

    else:
      for di in range(len(label)):
        decoder_output=decoder(decoder_input,decoder_hidden,encoder_outputs)

        topv,topi=decoder_output.topk(1) #(값이 큰 value를 k개 value, value의 위치값 인덱스)
        decoder_input=topi.squeeze().detach() #gradient 연산 작업에서 제외

        target=torch.tensor(label[di],dtype=torch.long).to(device)
        target=torch.unsqueeze(target,dim=0).to(device)
        loss+=nn.CrossEntropyLoss()(decoder_output,target)

        if decoder_input.item()==1:
          break

    total_loss+=loss.item()/len(dataset)
    iterator.set_description(f'epoch:{epoch+1} loss:{total_loss}')
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
  torch.save(encoder.state_dict(),'attn_enc.pth')
  torch.save(decoder.state_dict(),'attn_dec.pth')

  0%|          | 0/5701 [00:00<?, ?it/s]<ipython-input-11-334a5207a418>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data=torch.tensor(data,dtype=torch.long).to(device)
<ipython-input-11-334a5207a418>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label=torch.tensor(label,dtype=torch.long).to(device)
<ipython-input-11-334a5207a418>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target=torch.tensor(label[di],dtype=torch.long).to(device)
epoch:1 loss:0.003116619710315008:   0%|          | 1/5701 [00:01<2:17:58,  1.45s/it]<ipython

In [12]:
encoder.load_state_dict(torch.load('attn_enc.pth',map_location=device))
decoder.load_state_dict(torch.load('attn_dec.pth',map_location=device))
idx=random.randint(0,len(dataset))
print(len(dataset))
print(idx)
input_sentence=dataset.eng_corpus[idx]
pred_sentence=""

data,label=dataset[idx]
data=torch.tensor(data,dtype=torch.long).to(device)
label=torch.tensor(label,dtype=torch.long).to(device)

encoder_hidden=torch.zeros(1,1,64).to(device)
encoder_outputs=torch.zeros(11,64).to(device)

for ei in range(len(data)):
  encoder_output,encoder_hidden=encoder(data[ei],encoder_hidden)
  encoder_outputs[ei]=encoder_output[0,0]

decoder_intput=torch.tensor([[0]]).to(device)
decoder_hidden=encoder_hidden

for di in range(11):
  decoder_output=decoder(decoder_input, decoder_hidden, encoder_outputs)
  topv, topi=decoder_output.topk(1)
  decoder_input=topi.squeeze().detach()

  if decoder_input.item()==1:
    break

  pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "

print(input_sentence)
print(pred_sentence)


<ipython-input-12-ec365707ca5d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load('attn_enc.pth',map_location=device))


5701
1447
whatre you doing



<ipython-input-12-ec365707ca5d>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  decoder.load_state_dict(torch.load('attn_dec.pth',map_location=device))
